# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::100] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset = get_dataset([3,5,7,9], path="collected_data/rl_det_small.txt")

[ 0.00000000e+00  7.95731469e+08 -8.17891077e-02 -1.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.09713430e-01 -2.63658359e-01  6.00000000e-01]
Read chunk # 4 out of 10000
[ 0.00000000e+00  7.95731469e+08  1.24610892e-01  2.40000469e-03
 -7.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.20016566e-01  3.79282423e-01 -6.00000000e-01]
Read chunk # 6 out of 10000
[ 0.00000000e+00  7.95731469e+08 -9.01891077e-02  1.08000047e-02
  3.99986580e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.17973088e-02 -2.40776052e-01  6.00000000e-01]
Read chunk # 8 out of 10000
[ 0.00000000e+00  7.95731469e+08  6.91108923e-02 -5.99999531e-03
 -6.00001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.42355434e-01  2.22081792e-01 -6.00000000e-01]
Read chunk # 10 out of 10000
torch.Size([111080, 6])


In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -4.1227446,
 'std': 2.4676569,
 'min': -12.578855,
 'max': 0.0,
 'histogram': (array([ 26,   9,   7,   7,   8,   7,  10,  13,  27,  54,  56,  73, 109,
          84, 186, 148, 124,  83,  67,  13]),
  array([-12.578855 , -11.949912 , -11.320969 , -10.692026 , -10.063084 ,
          -9.434141 ,  -8.805199 ,  -8.176255 ,  -7.5473127,  -6.9183702,
          -6.2894273,  -5.6604843,  -5.031542 ,  -4.4025993,  -3.7736564,
          -3.1447136,  -2.515771 ,  -1.8868282,  -1.2578855,  -0.6289427,
           0.       ], dtype=float32))}

In [5]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [12]:
from d3rlpy.algos import TD3PlusBC

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

model = TD3PlusBC(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [7]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

-0.08131087201637716


In [8]:
%load_ext tensorboard
%tensorboard --logdir runs

In [9]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=5, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-07 19:43.22 [debug    ] RoundIterator is selected.
2022-04-07 19:43.22 [info     ] Directory is created at d3rlpy_logs/CQL_20220407194322
2022-04-07 19:43.22 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-07 19:43.22 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-07 19:43.22 [warning  ] Skip building models since they're already built.
2022-04-07 19:43.22 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220407194322/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 19:43.45 [info     ] CQL_20220407194322: epoch=1 step=343 epoch=1 metrics={'time_sample_batch': 0.000332894200138726, 'time_algorithm_update': 0.06332637200202608, 'temp_loss': 4.549272245290328, 'temp': 0.9833125291343333, 'alpha_loss': -9.526565534901689, 'alpha': 1.014112019677899, 'critic_loss': 10.390127244913195, 'actor_loss': 0.17096432040475373, 'time_step': 0.06374213021281162, 'td_error': 1.640563069486044, 'init_value': -0.2725396454334259, 'ave_value': -0.2704807368589359} step=343
2022-04-07 19:43.45 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220407194322/model_343.pt


Epoch 2/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 19:44.10 [info     ] CQL_20220407194322: epoch=2 step=686 epoch=2 metrics={'time_sample_batch': 0.000326751620011844, 'time_algorithm_update': 0.07104923704275237, 'temp_loss': 2.526219411077027, 'temp': 0.956864971113622, 'alpha_loss': 4.13621229787273, 'alpha': 1.0176574339324462, 'critic_loss': -3.2920629723600325, 'actor_loss': 3.1353582501758988, 'time_step': 0.07146202371002286, 'td_error': 4.960642614457846, 'init_value': -1.62131667137146, 'ave_value': -1.6176998761369101} step=686
2022-04-07 19:44.10 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220407194322/model_686.pt


...

Epoch 4/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 19:45.02 [info     ] CQL_20220407194322: epoch=4 step=1372 epoch=4 metrics={'time_sample_batch': 0.00034571597596994296, 'time_algorithm_update': 0.07863425443888405, 'temp_loss': 0.44537788994458255, 'temp': 0.9308691212456706, 'alpha_loss': 13.288155922736788, 'alpha': 0.9422573655756847, 'critic_loss': -10.698442213041789, 'actor_loss': 7.801613221015597, 'time_step': 0.07906752425102034, 'td_error': 7.775450190520213, 'init_value': -4.5973310470581055, 'ave_value': -4.594196638820768} step=1372
2022-04-07 19:45.02 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220407194322/model_1372.pt


Epoch 5/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-07 19:45.33 [info     ] CQL_20220407194322: epoch=5 step=1715 epoch=5 metrics={'time_sample_batch': 0.00034187695027787903, 'time_algorithm_update': 0.09032315276454558, 'temp_loss': 0.03165825267265342, 'temp': 0.9276943024323911, 'alpha_loss': 14.31788788453483, 'alpha': 0.9025450887554937, 'critic_loss': -10.340598141139171, 'actor_loss': 9.92209205126971, 'time_step': 0.09075341836356561, 'td_error': 9.699289034171423, 'init_value': -7.284750938415527, 'ave_value': -7.281605792852697} step=1715
2022-04-07 19:45.33 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220407194322/model_1715.pt


[(1,
  {'time_sample_batch': 0.000332894200138726,
   'time_algorithm_update': 0.06332637200202608,
   'temp_loss': 4.549272245290328,
   'temp': 0.9833125291343333,
   'alpha_loss': -9.526565534901689,
   'alpha': 1.014112019677899,
   'critic_loss': 10.390127244913195,
   'actor_loss': 0.17096432040475373,
   'time_step': 0.06374213021281162,
   'td_error': 1.640563069486044,
   'init_value': -0.2725396454334259,
   'ave_value': -0.2704807368589359}),
 (2,
  {'time_sample_batch': 0.000326751620011844,
   'time_algorithm_update': 0.07104923704275237,
   'temp_loss': 2.526219411077027,
   'temp': 0.956864971113622,
   'alpha_loss': 4.13621229787273,
   'alpha': 1.0176574339324462,
   'critic_loss': -3.2920629723600325,
   'actor_loss': 3.1353582501758988,
   'time_step': 0.07146202371002286,
   'td_error': 4.960642614457846,
   'init_value': -1.62131667137146,
   'ave_value': -1.6176998761369101}),
 (3,
  {'time_sample_batch': 0.000331777872914128,
   'time_algorithm_update': 0.0679256

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [10]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_det_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08 -1.03891077e-02 -1.41999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78778459e-03 -1.34615461e-02  4.84073546e-02]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.23311010e-02 -1.64283998e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08  7.01089229e-03 -4.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21623335e-01 -2.86362315e-02 -8.00043364e-02]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01 -1.37999953e-02
  7.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.76352555e-01 -3.26280816e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-07 19:45.34 [debug    ] RoundIterator is selected.
2022-04-07 19:45.34 [info     ] Directory is created at d3rlpy_logs/FQE_2022040719453

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

/Users/hetankevin/miniforge3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hetankevin/miniforge3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2022-04-07 19:45.37 [info     ] FQE_20220407194534: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00015500764911973122, 'time_algorithm_update': 0.0021512693044536475, 'loss': 0.0008598907446906451, 'time_step': 0.0023798494512779567, 'init_value': -0.3687102198600769, 'ave_value': -0.36885178882223535, 'soft_opc': nan} step=878


2022-04-07 19:45.37 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194534/model_878.pt


...

2022-04-07 19:47.12 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194534/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:47.14 [info     ] FQE_20220407194534: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.00012273913363932478, 'time_algorithm_update': 0.001632396886995007, 'loss': 0.015797932613698454, 'time_step': 0.0018077195369572737, 'init_value': -1.2501708269119263, 'ave_value': -1.2492717514301808, 'soft_opc': nan} step=43900


2022-04-07 19:47.14 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194534/model_43900.pt


[(1,
  {'time_sample_batch': 0.00015500764911973122,
   'time_algorithm_update': 0.0021512693044536475,
   'loss': 0.0008598907446906451,
   'time_step': 0.0023798494512779567,
   'init_value': -0.3687102198600769,
   'ave_value': -0.36885178882223535,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.0001387566260161867,
   'time_algorithm_update': 0.0018661529979836153,
   'loss': 0.0017972966269422025,
   'time_step': 0.0020636958250423772,
   'init_value': -0.4670436978340149,
   'ave_value': -0.46696345135451456,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00015332378396140688,
   'time_algorithm_update': 0.0021614496148528706,
   'loss': 0.0028082218235471962,
   'time_step': 0.0023881615158637183,
   'init_value': -0.6038172245025635,
   'ave_value': -0.603597512889957,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00014146884101398443,
   'time_algorithm_update': 0.001892091201486783,
   'loss': 0.004026786944634962,
   'time_step': 0.0020962252975324836,
   

In [11]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_stoch_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08  1.39310892e-01  1.82000047e-02
 -1.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.50230117e-02  3.69851546e-01 -6.00000000e-01]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.15389108e-01  1.64000047e-02
 -8.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08831172e-01 -2.48178665e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.28589108e-01  1.20000047e-02
  1.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.75212759e-03 -3.52719043e-01  6.00000000e-01]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01  7.00000469e-03
 -8.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.65974295e-01 -2.19295880e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-07 19:47.15 [debug    ] RoundIterator is selected.
2022-04-07 19:47.15 [info     ] Directory is created at d3rlpy_logs/FQE_2022040719471

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:47.17 [info     ] FQE_20220407194715: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00012011843010070384, 'time_algorithm_update': 0.0016416873367066263, 'loss': 0.0005790919066507191, 'time_step': 0.0018213905886256885, 'init_value': -0.2699924111366272, 'ave_value': -0.2707972494631286, 'soft_opc': nan} step=878


2022-04-07 19:47.17 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194715/model_878.pt


Epoch 2/50:   0%|          | 0/878 [00:00<?, ?it/s]

...

2022-04-07 19:49.12 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194715/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-07 19:49.14 [info     ] FQE_20220407194715: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.00013469672148754493, 'time_algorithm_update': 0.0020765655404614424, 'loss': 0.014310937399106917, 'time_step': 0.0022711433419334045, 'init_value': -1.0202535390853882, 'ave_value': -1.0242416432717358, 'soft_opc': nan} step=43900


2022-04-07 19:49.14 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220407194715/model_43900.pt


[(1,
  {'time_sample_batch': 0.00012011843010070384,
   'time_algorithm_update': 0.0016416873367066263,
   'loss': 0.0005790919066507191,
   'time_step': 0.0018213905886256885,
   'init_value': -0.2699924111366272,
   'ave_value': -0.2707972494631286,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00012525257597208566,
   'time_algorithm_update': 0.0017642578394374978,
   'loss': 0.0016421610482464442,
   'time_step': 0.0019463102475385731,
   'init_value': -0.4241451919078827,
   'ave_value': -0.42590119195731213,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00012309458912912425,
   'time_algorithm_update': 0.0017062292555198582,
   'loss': 0.0032858520809319765,
   'time_step': 0.0018910036543235691,
   'init_value': -0.5432124137878418,
   'ave_value': -0.5457452573710693,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00012767043363532063,
   'time_algorithm_update': 0.0018168163734036318,
   'loss': 0.005045686625876784,
   'time_step': 0.0020041721013792555,
 